## Testing Agent on new emails

## Launch Notebooks

In [0]:
%run ./step_3_Agent_Graph_inMemory

In [0]:
#%run ./step_0_Environment

In [0]:
#%run ./step_0_Variables

## Set up ML flow Experiment

In [0]:
experiment_name_ = "Email_Classificator_Agent_20260127_final"

In [0]:
import mlflow
experiment_path = experiment_dir_ + experiment_name_
experiment = mlflow.get_experiment_by_name(experiment_path)

if experiment is not None:
    experiment_id = experiment.experiment_id
else:
    experiment_id = mlflow.create_experiment(name=experiment_path)

## Run Tests

In [0]:
## Save test cases
spark.sql("USE CATALOG "+catalog_)
spark.sql("USE SCHEMA "+schema_)
df_test_emails = spark.table("classificator_agent_emails")

In [0]:
## Create helper function
from pyspark.sql.functions import *
from langchain_core.messages import HumanMessage

# Extract test case from the dataframe with examples
def launch_agent(id_, run_name_ = "Test new emails 27 Jan 2026"):
  """ Launch agent on the email associated to the record id_"""

  ## Prepare request
  test_record = df_test_emails.filter(col("Email_Id") == id_).first()
  message_ = f"We received from: {test_record.Email_Sender} the following email: {test_record.Email_Body}"
  print("\n\n***************************************\n")
  print(":: Testing agent on record %d:\n" % id_)
  display(message_)

  ## Invoke the Agent
  config_ = {"configurable": {"thread_id": id_}} # Not required if we're not using memory
  mlflow.langchain.autolog()
  with mlflow.start_run(experiment_id=experiment_id, run_name=run_name_):
    request = [
      HumanMessage(content = message_)
    ]
    messages = react_email_classifier.invoke({"messages": request}, config_)

    for m in messages['messages']:
        m.pretty_print()

    return messages

In [0]:
## Launch Tests
test_email_ids = [row.Email_Id for row in df_test_emails.select("Email_Id").collect()]
results_ = []

for id_ in test_email_ids:
  result = launch_agent(id_)
  results_.append(result)

In [0]:
## Progragmatically extract Labels
final_labels_ = []
for res_ in results_:
  final_label = res_['final_output'].Label
  final_labels_.append(final_label)
  print(final_label)

### Verify conext info

In [0]:
%sql
-- Available Context: Customer Details
SELECT classificator_agent_customer_retriever('david.nguyen@example.com') AS customer_details;

In [0]:
%sql
-- Available Context: Order Details
SELECT classificator_agent_order_retriever(6) AS order_details;

In [0]:
%sql
-- Available Context: Ticket Details
SELECT classificator_agent_ticket_retriever(6) AS ticket_details;